## Store the memory in DynamoDB
We can store previous conversation in DynamoDB to preserve state in our application. We do this with the help of LangChains "DynamoDBChatMessageHistory" module.

In [ ]:
session = boto3.session.Session(profile_name="bedrock")
dynamodb = session.resource("dynamodb")
table_name = "chatbot-memory-sessions"

table = dynamodb.create_table(
    TableName = table_name,
    KeySchema = [{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions = [{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode = "PAY_PER_REQUEST",
)

#### LangChain setup
We import the module, configure our memory and provide it a session id.

In [ ]:
bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name="us-east-1",
)

model_kwargs = {
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_p": 0.5
}

chat = Bedrock(
    credentials_profile_name="bedrock",
    model_id="anthropic.claude-v2",
    model_kwargs=model_kwargs,
    client=bedrock_runtime,
)

In [ ]:
import uuid
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
session_id = uuid.uuid4()
chat_history = DynamoDBChatMessageHistory(table_name=table_name, session_id=session_id)
memory = ConversationBufferMemory(
    memory_key="history",
    chat_memory=chat_history,
    ai_prefix="Assistant",
    human_prefix="Human"
)

conversation = ConversationChain(
    llm=chat, verbose=True, memory=memory
)

#### Query

In [ ]:
conversation.predict(input="Hi there!")
conversation.predict(input="Can you explain why the grass is green?")
conversation.predict(input="Okay great, so what do you mean with the expression "the grass is always greener on the other side"?)

memory.load_memory_variables({}) 

print(table.item_count)